In [2]:
import requests
import pandas as pd
from pprint import pprint
from operator import itemgetter
import json
import time
from datetime import datetime

from functools import reduce

from bs4 import BeautifulSoup

import random

# Добыча лайков

In [1]:
url = "https://www.instagram.com/p/Btndd-hhSF2/"

In [ ]:
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains

# настройки для запуска в фоновом режиме
# opts = Options()
# opts.set_headless()
# assert opts.headless

In [ ]:
# with open("patterns.json", "w") as f:
#     json.dump({"liked_by_user": class_pattern, "liked_by_button": 'zV_Nj'}, f)

## Это работает

In [ ]:
with open("patterns.json") as f:
    patterns = json.load(f)

In [ ]:
browser = Firefox()
browser.get(url)
browser.find_element_by_xpath("//a[@class='{}']".format(patterns['liked_by_button'])).click()

In [ ]:
frame = browser.find_element_by_xpath("/html/body/div[3]/div/div[2]/div")
part_users = frame.text.split("\n")
[part_users[i] for i in range(0, len(part_users), 3)]

# Скроллинг

In [ ]:
browser.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", frame)  # рабочий вариант

In [ ]:
data = [i.text for i in browser.find_elements_by_xpath("//div[@class='{}']".format(patterns["liked_by_user"]))]
SCROLL_PAUSE_TIME = 0.5

# Get scroll height
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    data.extend([i.text for i in browser.find_elements_by_xpath("//div[@class='{}']".format(patterns["liked_by_user"]))])
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

# Сбор данных постов, найденных по тэгу

Структура json
```python
["graphql"]["hashtag"]["edge_hashtag_to_media"]["page_info"]
```

In [ ]:
def get_json(path, next_page=None):
    """
    Returns json from url https://www.instagram.com/ + path
    
    path: str, part of url that follows https://www.instagram.com/
    next_page: str, link to next page similar to linked list, 
    got from 'cursor' field in response json
    
    returns: dict"""
    
    url = "https://www.instagram.com/" + path
    params = {
        "__a": 1
    }
    if next_page:
        params["max_id"] = {next_page}
        
    r = requests.get(url, params=params)
    try:
        return r.json()
    except json.decoder.JSONDecodeError as e:
        print(r)
        raise json.decoder.JSONDecodeError

In [ ]:
def pages_by_tag(tag, max_pages, retries=5):
    """Instagram pages generator, yields n pages found by hash tag,
    where n is less or equal than max_pages
    
    tag: str, instagram hash tag - a string without '#' symbol
    max_pages: int, max n of pages to yield
    
    yields: dict"""
    
    next_page = None
    page_n = 0
    path = f"explore/tags/{tag}/"
    retries_counter = 0
    
    while page_n <= max_pages:
        page_n += 1
        data = get_json(path, next_page)
        print(f"Got page {page_n}")
        
        try:
            page_info = data["graphql"]["hashtag"]["edge_hashtag_to_media"]["page_info"]
            retries_counter = 0
        except KeyError:
            if retries_counter <= retries:
                time.sleep(1)
                print(f"Attempt #{retries_counter} on page {page_n}...")
                retries_counter += 1
#             print(next_page)
#             print(data)
                continue
            else:
                print(f"Unable to get page data\ncursor: {next_page}")
                break
        
        if page_info["has_next_page"]:
            next_page = page_info["end_cursor"]
        else:
            print("No more pages")
            break
        
        yield data

In [ ]:
#использовать как декоратор
def parse_page(page_json, keys, verbose=False):
    """Parses instagram page json
    page_json: dict, json got from server response
    keys: iterable, list of fields to retrieve from page_json
    
    returns: dict"""
    
    parsed_data = {}
    page_posts = page_json["graphql"]["hashtag"]["edge_hashtag_to_media"]["edges"]
    for post in page_posts:
#         print(1)
        raw_post = post["node"]
        parsed_post = {}
        for key in keys:
            try:
                parsed_post[key] = raw_post[key]
            except KeyError:
                try:
                    parsed_post[key] = raw_post["edge_media_to_caption"]["edges"][0]["node"][key]
                except KeyError:
                    if verbose:
                        print(f"Key {key} was not found")
                except IndexError as e:
                    if verbose:
                        print(f"Key {key} was not found, reason:\n{e}")
        parsed_data[raw_post["shortcode"]] = parsed_post
    return parsed_data

In [ ]:
def get_tag_data(tag, keys, max_pages=10):
    """Collects data from instagram posts found by tag
    
    tag: str, instagram hash tag without '#' symbol
    keys: fields from instagram response json to retrieve
    max_pages: int, max n of pages to parse
    
    returns: dict, with pairs <post_shortcode>: {<post_data>}"""
    
    tag_data = {}
    for page in pages_by_tag(tag, max_pages):
        time.sleep(2)
        page_data = parse_page(page, keys)
        tag_data.update(page_data)
    return tag_data

In [ ]:
tag = "мастеркласс"
keys = [
    "text",
    "taken_at_timestamp",
    "edge_liked_by",
    "owner",
    "is_video"
]
# stop_date = datetime.strptime("01.01.2019", "%d.%M.%Y").timestamp()

In [ ]:
get_page_data(next(pages_by_tag(tag, 1)), keys)

In [ ]:
data = get_tag_data("мастеркласс", keys, max_pages=300)

In [ ]:
data_pd = pd.DataFrame.from_dict(data, orient='index')

In [ ]:
data_pd.reset_index(inplace=True)

In [ ]:
data_pd.to_csv("тэг_мастеркласс_300_страниц.csv")

In [ ]:
data_pd["taken_at_timestamp"].map(lambda x: datetime.fromtimestamp(x))

In [ ]:
# html class for liked by
liked_by_class = "pbNvD fPMEg"

In [ ]:
r = requests.get("https://www.instagram.com/p/Bu1eDsZDMSx/liked_by/")
soup = BeautifulSoup(r.content, "html.parser")